In [44]:
import pandas as pd
import numpy as np
from numpy  import array
import librosa
import os
import ntpath
import pathlib
from pathlib import Path
import pickle
from tinytag import TinyTag
from pydub import AudioSegment
from pydub.utils import make_chunks
from itertools import tee, islice, chain
from math import atan2,degrees


In [45]:
df = pd.read_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/audio_information_april_6.pkl")

In [46]:
# Determines the angle of a straight line drawn between point one and two.
def getAngleBetweenTwoPoints(p1x,p1y,p2x,p2y):
    xDiff = p2x - p1x
    yDiff = p2y - p1y
    angle = degrees(atan2(yDiff, xDiff))
    if(angle >= 0):
        return angle
    else:
        return (angle+360)
def getPreviousAndNext(iterable_data):
    prevs, items, nexts = tee(iterable_data, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)
def getArrayOfAnglesBetweenConsecutivepointsInArray(iterable_data):
    xx = 1
    yy = 2
    np_arr =  np.array([])
    for previous, item, nxt in getPreviousAndNext(iterable_data):
        if(nxt == None):
            np_arr = np.append(np_arr,0)
        else:
            tempvar = getAngleBetweenTwoPoints(xx,item,yy,nxt)
            np_arr = np.append(np_arr,tempvar)
    return np_arr

In [47]:
def getVocalPercentage(full,background,foreground):
    m =full/(background+foreground)
    full_new = full/m
    return (foreground/full_new)*100

In [48]:
def getAnglesBetweenConsecutiveBeats(beat_times,onset_envelope):
    beat_strength = []
    for j in  beat_times:
        beat_strength.append(onset_envelope[j])
    return  getArrayOfAnglesBetweenConsecutivepointsInArray(beat_strength)


In [49]:
def generateDataFrameWithHeader(header):
    return pd.DataFrame(columns=header)

In [50]:
def getFeatureList(i):
    full_spectrum_mean = df['full_spectrum_mean'][i]
    instruments_mean = df['instruments_mean'][i]
    vocals_mean = df['vocals_mean'][i]
    audio_waveform = df['audio_waveform'][i]
    onset_envelope = df['onset_strength'][i]
    beat_times = df['beat'][i]
    tempo = df['tempo'][i]
    
    featurelist = []
    for m in metadata_arr:
        featurelist.append(df[m][i])
    
    featurelist.append(beat_times)
    featurelist.append(tempo)
    featurelist.append(getAnglesBetweenConsecutiveBeats(beat_times,onset_envelope))
    featurelist.append(getVocalPercentage(full_spectrum_mean,instruments_mean,vocals_mean))
    return featurelist

In [51]:
def addFeatureToDataFrame(count_row):
    i = 1
    while (i<=count_row):
        df_new.loc[i] = getFeatureList(i)
        i += 1

In [52]:
header = np.ndarray(0) 
metadata  = 'filename artist duration albumartist bitrate comment filesize genre title track samplerate track_total year'
features = 'bpm tempo angles_between_consecutive_beats vocal_percentage'
metadata_arr = metadata.split()
features_arr = features.split()
header = np.append(header,metadata_arr)
header = np.append(header,features_arr)
df_new = generateDataFrameWithHeader(header)
addFeatureToDataFrame(df.shape[0])    
    
    
    

In [53]:
df_new.head()

,filename,artist,duration,albumartist,bitrate,comment,filesize,genre,title,track,samplerate,track_total,year,bpm,tempo,angles_between_consecutive_beats,vocal_percentage
1,Bebe Rexha - Drinking About You (The Chainsmok...,Bebe Rexha,217.034914,The Chainsmokers Bebe Rexha,320.0,,8712257,Electronic,Drinking About You (The Chainsmokers Remix),None,44100,None,2013,b'\x80\x04\x95\xb2\x07\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95k\x00\x00\x00\x00\x00\x00\x00\x8...,"[270.2246881796245, 0.0, 0.0, 0.0, 89.55238582...",7.632351


In [54]:
count_row_dfNew = df_new.shape[0]  # gives number of row count
print(count_row_dfNew)

1


In [55]:
df_new.to_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/audio_information_april_6_Phase_2.pkl",protocol = -1)
